In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 2nd Place Solution for UCSD-Winter-2025-DSC148 Air BnB Price Prediction Contest

##### Contest: https://www.kaggle.com/competitions/ucsd-winter-2025-dsc-148/leaderboard

##### Goal: Build a model that predicts the price of Air BnB's in New York City with the lowest RMSE.

##### Background: This competition is part of the course curriculum at UCSD for the class DSC 148: Data Mining. 109 students took part in this competition. A portion of our final class grade depended on the ranking in this competition. 

##### Outcome: 2nd out of 109 student; RMSE - 82.40811. First place RMSE - 82.32490.

##### Approach: Use XGBoost with as much relevant data as possible. Use extensive hyperparameter tuning and regularization to avoid overfitting and improve generalization.

In [212]:
# Load Data
df = pd.read_csv("train.csv", low_memory=False)

In [213]:
# One hot encode the ammenities feature
amenities = set([string.strip("\"") for i in range(df.shape[0]) 
                 for string in df["amenities"].iloc[i][1:-1].split(',')])
amenities.remove('')
amenities.remove('translation missing: en.hosting_amenity_49')
amenities.remove('translation missing: en.hosting_amenity_50')
amenities = list(amenities)

amenities_df = pd.concat(
    [df["amenities"]
     .apply(lambda x: 1 if amenity in 
            [string.strip("\"") for string in x[1:-1].split(',')] 
            else 0)
     .rename(amenity) for amenity in amenities], axis = 1
)

df = pd.concat([df, amenities_df], axis = 1)

In [214]:
# Encodes text features using Latent Dirichlet Allocation

# Combine all text features into a single feature
text_labels = ['name', 'summary', 'space', 'description', 
                 'neighborhood_overview', 'notes', 'transit', 'access', 
                 'interaction', 'house_rules']
df['text'] = df[text_labels].fillna('').agg(' '.join, axis=1)

# Convert text into a document-term matrix
vectorizer = CountVectorizer(max_features=10000, stop_words='english')
doc_term_matrix = vectorizer.fit_transform(df['text'].fillna(''))

# Fit LDA model
n_components = 20
lda = LatentDirichletAllocation(n_components=n_components, random_state = 25)
topic_matrix = lda.fit_transform(doc_term_matrix)

# Add topic probabilities to the dataset
df[[f'topic_{i}' for i in range(n_components)]] = topic_matrix

In [215]:
X = df.drop(columns=["price"])
y = df["price"]

In [ ]:
# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Number of top words to display per topic
n_top_words = 10

# Print top words for each topic
for topic_idx, topic in enumerate(lda.components_):  # `lda.components_` contains topic-word distribution
    top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: tv, bed, kitchen, apartment, coffee, bedroom, wifi, microwave, queen, size
Topic 1: room, train, use, house, clean, people, like, kitchen, place, just
Topic 2: minutes, train, manhattan, away, bus, jfk, airport, house, neighborhood, min
Topic 3: new, apartment, brand, renovated, york, fully, newly, high, furnished, appliances
Topic 4: apartment, city, new, nyc, east, park, york, away, home, stay
Topic 5: private, floor, garden, outdoor, deck, terrace, duplex, amenities, new, views
Topic 6: building, room, apartment, kitchen, features, hotel, bedroom, unit, floors, appliances
Topic 7: guests, room, guest, check, space, home, clean, private, stay, time
Topic 8: room, kitchen, living, bathroom, bedroom, private, apartment, shared, bed, access
Topic 9: village, east, apartment, soho, west, square, restaurants, walk, heart, building
Topic 10: walk, central, building, apartment, studio, subway, park, minute, located, min
Topic 11: square, chelsea, apartment, away, union, times, subw

In [217]:
# Make predictions on the test set
def predict_test(model):
    
    test = pd.read_csv("test.csv", low_memory=False)

    # One Hot encode ammenities
    amenities_df = pd.concat(
        [test["amenities"]
        .apply(lambda x: 1 if amenity in 
                [string.strip("\"") for string in x[1:-1].split(',')] 
                else 0)
        .rename(amenity) for amenity in amenities], axis = 1
    )

    test = pd.concat([test, amenities_df], axis = 1)

    # Encode text feautures using pre-trained LDA model

    # Aggregate text features into a single feature
    test['text'] = test[text_labels].fillna('').agg(' '.join, axis=1)

    # Transform the test set using the already trained vectorizer
    doc_term_matrix_test = vectorizer.transform(test['text'].fillna(''))

    # Get topic probabilities using the trained LDA model
    topic_matrix_test = lda.transform(doc_term_matrix_test)

    # Add topic probabilities to the test dataset
    test[[f'topic_{i}' for i in range(n_components)]] = topic_matrix_test

    # Predict and save the test set
    preds = model.predict(test)
    preds = 5 * np.round(preds / 5)
    output = pd.concat([test["id"], pd.Series(preds)], axis = 1)
    output = output.rename(columns={"id":"Id", 0:"Predicted"})
    output.to_csv("predictions.csv", index=False)
    print("Output written.")

In [218]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df['price']))
fig.update_layout(title=dict(text='Price Distribution', x=0.5),
                  xaxis = dict(range=(0,1000)),
                  height = 400, width = 600,
                  margin=dict(l=50, r=50, t=50, b=50))
fig.write_image('price_dist.png', scale = 4)
fig.show()

In [220]:
df[df['price'] % 5 == 0].shape[0] / df.shape[0] * 100

71.98998151350706

In [221]:
# Drops irrelevant columns
def choose_columns(X, drop_cols):
    
    return X.drop(columns=drop_cols)

In [ ]:
# Cleans zip code feature
def clean_zip(X):
    X = X.copy()
    
    X['zipcode'] = (pd.Series(X['zipcode'].apply(lambda x: str(x)[:5]))
                    .apply(lambda x: x if x.isnumeric() else 'other'))

    return X

In [224]:
# Transforms categorical variables
def clean_numerical(X):
    X = X.copy()

    # Clean and fill nan with 0
    X['extra_people'] = (X["extra_people"]
                         .fillna('$0.0')
                         .apply(lambda x: float(x[1:])))

    # Clean and keep nan
    X["host_response_rate"] = (X["host_response_rate"]
                               .apply(lambda x: int(str(x)[:-1]) 
                                      if str(x) != 'nan' else np.nan))

    # Fill bedrooms nan with median of 'accommodates'
    X["bedrooms"] = (
        X["bedrooms"]
        .fillna(X.groupby("accommodates")["bedrooms"].transform("median"))
    )

    # Fill beds nan with median of 'accommodates'
    X["beds"] = (
        X["beds"]
        .fillna(X.groupby("accommodates")["beds"].transform("median"))
    )

    # Fill bathrooms nan with median of 'accommodates'
    X["bathrooms"] = (
        X["bathrooms"]
        .fillna(X.groupby("accommodates")["bathrooms"].transform("median"))
    )

    # Extract year
    X['host_since'] = pd.to_datetime(X['host_since']).dt.year

    return X

In [227]:
categorical_features = [
    "neighbourhood_group_cleansed",
    "neighbourhood_cleansed",
    "zipcode", 
    "property_type", 
    "room_type", 
    'bed_type',
    "cancellation_policy",
    "host_response_time",
    "host_is_superhost",
    'host_identity_verified', 
    'instant_bookable',
    ]

numerical_features = [ 
    "accommodates", 
    "bathrooms",
    "bedrooms", 
    "beds",
    'square_feet',
    "guests_included",
    'extra_people',
    'review_scores_rating', 
    'review_scores_accuracy',
    'review_scores_cleanliness', 
    'review_scores_checkin',
    'review_scores_communication', 
    'review_scores_location',
    'review_scores_value',
    "host_listings_count",
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'host_since',
    'host_response_rate',
    ]

potential_features = ["host_id", 
                      'host_location',  
                      'host_verifications', 
                      'host_has_profile_pic',
                      'require_guest_profile_picture',
                      'require_guest_phone_verification',
                      'city',
                      'first_review', 
                      'last_review',
                      'host_neighbourhood',
                      'host_about',
                      'calculated_host_listings_count',
                      ]

text_features = ['name', 'summary', 'space', 'description', 
                 'neighborhood_overview', 'notes', 'transit', 'access', 
                 'interaction', 'house_rules']

text_topics = [f'topic_{i}' for i in range(n_components)]

useless_features = ["id", "experiences_offered", "host_name", 
                    "host_acceptance_rate", 'state', 'market', 'country_code',
                    'country', "is_business_travel_ready"]

# Handled once manually in test & train dataset initialization due to computation time
special_features = ['amenities', 'text'] 

drop_cols = potential_features + text_features + useless_features + special_features

In [228]:
# Explicity define the categories for one hot encoding
categories = [
    list(clean_zip(df)[feature].unique()) for feature in categorical_features
    ]

for category_lst in categories:
    if np.nan in category_lst:
        category_lst.remove(np.nan)
        category_lst.append(np.nan)

In [229]:
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore',categories=categories), 
               categorical_features),
    ('scaler', RobustScaler(), numerical_features),],
    remainder='passthrough'
)

pipe = Pipeline([
    ('zip', FunctionTransformer(clean_zip)),
    ('drop', FunctionTransformer(choose_columns, 
                                 kw_args={"drop_cols":drop_cols})),
    ('clean_numerical', FunctionTransformer(clean_numerical)),
    ('preprocessor', preprocessor),
    ('model', XGBRegressor())
])

pipe.set_params(**{
 'model__learning_rate': 0.025,
 'model__max_depth': 9,
 'model__min_child_weight': 1,
 'model__min_split_loss': 6.5,
 'model__n_estimators': 1100,
 'model__reg_alpha': 0,
 'model__reg_lambda': 18,
 'model__subsample': 0.5,
 'model__colsample_bytree': 1,
 'model__tree_method': 'hist'
})

print(round(-cross_val_score(pipe, X, y, cv=10 ,scoring="neg_root_mean_squared_error", n_jobs = 6).mean(),4))


80.6054


In [171]:
pipe.fit(X, y)
predict_test(pipe)

Output written.


In [356]:
param_grid = {
    'model__learning_rate': [0.03],  # Step size shrinkage
    'model__max_depth': [8],  # Tree depth
    'model__min_child_weight': [0.001, 0.05, 0.10],
    'model__min_split_loss':[5,6],
    'model__n_estimators': [750, 850],  # Number of boosting rounds
    'model__reg_alpha': [0],  # L1 regularization
    'model__reg_lambda': [15],  # L2 regularization
    'model__subsample': [0.5],  # Fraction of data used per boosting round   
}

# Define Grid Search
grid_search = GridSearchCV(pipe, param_grid, cv=10, scoring='neg_root_mean_squared_error', n_jobs=6, verbose=10)

grid_search.fit(X,y)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV 1/10; 1/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.001, model__min_split_loss=5, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 2/10; 1/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.001, model__min_split_loss=5, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 3/10; 1/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.001, model__min_split_loss=5, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 4/10; 1/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.001, model__min_split_loss=5, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 5/10; 1/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child

/opt/anaconda3/envs/dsc148/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 6/10; 7/12] END model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.05, model__min_split_loss=6, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5;, score=-80.651 total time=  50.7s
[CV 1/10; 8/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.05, model__min_split_loss=6, model__n_estimators=850, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 2/10; 8/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.05, model__min_split_loss=6, model__n_estimators=850, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5
[CV 7/10; 7/12] END model__learning_rate=0.03, model__max_depth=8, model__min_child_weight=0.05, model__min_split_loss=6, model__n_estimators=750, model__reg_alpha=0, model__reg_lambda=15, model__subsample=0.5;, score=-86.391 total time=  50.3s
[CV 3/10; 8/12] START model__learning_rate=0.03, model__max_depth=8, model__min_child_

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('clean_zip',
                                        FunctionTransformer(func=<function clean_zip at 0x3294680e0>)),
                                       ('drop',
                                        FunctionTransformer(func=<function choose_columns at 0x3746c8040>,
                                                            kw_args={'cat': ['neighbourhood_group_cleansed',
                                                                             'neighbourhood_cleansed',
                                                                             'zipcode',
                                                                             'property_type',
                                                                             'room_type',
                                                                             'cancellation_policy',
                                                                             'host_response_time',
                                                                             'host_is...
                                                     n_estimators=750,
                                                     n_jobs=None,
                                                     num_parallel_tree=None, ...))]),
             n_jobs=6,
             param_grid={'model__learning_rate': [0.03],
                         'model__max_depth': [8],
                         'model__min_child_weight': [0.001, 0.05, 0.1],
                         'model__min_split_loss': [5, 6],
                         'model__n_estimators': [750, 850],
                         'model__reg_alpha': [0], 'model__reg_lambda': [15],
                         'model__subsample': [0.5]},
             scoring='neg_root_mean_squared_error', verbose=10)

In [358]:
grid_search.best_params_

{'model__learning_rate': 0.03,
 'model__max_depth': 8,
 'model__min_child_weight': 0.001,
 'model__min_split_loss': 5,
 'model__n_estimators': 750,
 'model__reg_alpha': 0,
 'model__reg_lambda': 15,
 'model__subsample': 0.5}